In [8]:
import json
from util import sanitize_filename
import os
from collections import defaultdict
from llm import chat_openai
from datetime import datetime

In [29]:
import os

chunk_size = 4000 # How many characters to read per chunk in website text
overlap = 25 # How much overlap between chunks
MAX_TOKENS = 40000 # 10K -- Roughly 3K tokens, $0.10 per MAX_TOKENs, max tokens to read per key question -- 40K tokens is $0.40
search_query = "How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?"

In [3]:
# # Can be moved to util.py
# # Get the list of key questions
# def get_key_questions_list_string(search_query_file_safe):
#     file_path = f'autoscious_logs/{search_query_file_safe}/decompositions/improved_decomposition.json'

#     with open(file_path, 'r') as f:
#         question_decomposition = json.load(f)
#     key_questions = question_decomposition['key_drivers']['1']['hypotheses']['1']['key_questions']
#     print("\nkey_questions\n", key_questions)

#     numbered_key_questions_string = ""
#     for key, value in key_questions.items():
#         first_two_words = ' '.join(value.split())
#         numbered_key_questions_string += f'{int(key) - 1}. {first_two_words}\n'

#     print("\nnumbered_key_questions_string\n", numbered_key_questions_string)
#     return key_questions, numbered_key_questions_string

In [10]:
# 1) Get key questions
search_query_file_safe = sanitize_filename(search_query)
# key_questions_dict, key_questions_list_string = get_key_questions_list_string(search_query_file_safe)
# print("key_questions_dict", key_questions_dict)

# 2) create facts folder and subfolders
facts_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/facts'
if not os.path.exists(facts_folder_path):
    os.makedirs(facts_folder_path)
# for kq_idx in key_questions_dict:
#     kq_facts_folder_path = f'{facts_folder_path}/kq{int(kq_idx)-1}'
#     if not os.path.exists(kq_facts_folder_path):
#         os.makedirs(kq_facts_folder_path)
#     with open(f'{kq_facts_folder_path}/facts.txt', 'w') as f:
#         pass

In [83]:
# Can be broken to prompts.py and util.py
import os

def extract_facts_from_website_text(search_query_file_safe, key_question, website_title, website_text, website_url):
    seed_initial_question_decomposition_prompt = f'''
Key question: 
{key_question}

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. Format as a JSON.
```json
{{
  "<insert best answer>": "<insert quote>"
}}
```

Source text: {website_text}

Respond only with the output, with no explanation or conversation.
'''
    # Ask GPT the prompt
    print("seed_initial_question_decomposition_prompt", seed_initial_question_decomposition_prompt)
    res = chat_openai(seed_initial_question_decomposition_prompt, model="gpt-3.5-turbo")
    print("Extracted quotes: ", res[0])

    # Save the quote to the corresponding key question index file
    res_json = json.loads(res[0])
    for key, value in res_json.items():
        answer = key
        quote = value

        # Only log if there is a quote
        if quote:
          file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.txt'

          with open(file_name, 'a', encoding='utf-8') as f:
              f.write(answer + os.linesep)

          # Save the best answer and quote into a JSON for reference retrieval later
          json_file_name = f'autoscious_logs/{search_query_file_safe}/facts/facts.json'
          if os.path.exists(json_file_name):
            with open(json_file_name, 'r', encoding='utf-8') as json_file:
                data = json.load(json_file)
          else:
              data = {}

          # Update the dictionary and save it back
          data[answer] = quote.replace('/"', '"') + f"[{website_url}]"
          
          with open(json_file_name, 'w', encoding='utf-8') as json_file:
              json.dump(data, json_file, ensure_ascii=False, indent=4)

    return

In [80]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sentence_transformers import SentenceTransformer
# import re

# model = SentenceTransformer('all-MiniLM-L6-v2')  # or 'all-mpnet-base-v2'

# def get_most_relevant_chunks(key_question, text, CHUNK_SIZE):
#     # 1. Split text into chunks
#     chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

#     # 2. For each chunk, compute the cosine similarity with key_question
#     key_embedding = model.encode([key_question])
#     chunk_embeddings = model.encode(chunks)
    
#     similarities = [cosine_similarity(key_embedding.reshape(1, -1), chunk_embedding.reshape(1, -1))[0][0] for chunk_embedding in chunk_embeddings]
    
#     # 3. Sort chunks by similarity
#     sorted_chunks = [chunk for _, chunk in sorted(zip(similarities, chunks), key=lambda pair: pair[0], reverse=True)]

#     # 4. Return top chunks
#     return sorted_chunks

In [30]:
# import torch
# from transformers import AutoTokenizer, AutoModel
# from sklearn.metrics.pairwise import cosine_similarity
# import re

# tokenizer = AutoTokenizer.from_pretrained("allenai/specter")
# model = AutoModel.from_pretrained("allenai/specter")

# def embed_text_using_specter(texts):
#     encoded_input = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
#     with torch.no_grad():
#         model_output = model(**encoded_input)
#     embeddings = model_output.last_hidden_state.mean(dim=1)
#     return embeddings

# def get_most_relevant_chunks_specter(key_question, text, CHUNK_SIZE):
#     # 1. Split text into chunks
#     chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]

#     # 2. For each chunk, compute the cosine similarity with key_question
#     key_embedding = embed_text_using_specter([key_question])
#     chunk_embeddings = embed_text_using_specter(chunks)
    
#     # Note: Since the embeddings are in torch tensors, we'll use torch's cosine similarity function
#     similarities = [cosine_similarity(key_embedding, chunk_embedding.reshape(1, -1))[0][0] for chunk_embedding in chunk_embeddings]
    
#     # 3. Sort chunks by similarity
#     sorted_chunks = [chunk for _, chunk in sorted(zip(similarities, chunks), key=lambda pair: pair[0], reverse=True)]

#     # 4. Return top chunks
#     return sorted_chunks


In [81]:
from rank_bm25 import BM25Okapi
import re

def chunk_text(text: str, key_question: str, chunk_size: int, overlap: int = 0) -> list[str]:
    """
    Splits a text into overlapping chunks and ranks them using BM25 based on relevance to a key question.
    
    Args:
    - text (str): The source text.
    - key_question (str): The key question to rank the chunks by.
    - chunk_size (int): The size of each chunk.
    - num_chunk_samples (int): The number of top-ranked chunks to return.
    - overlap (int): The size of the overlap between chunks. Default is 0.
    
    Returns:
    - list[str]: The top-ranked chunks based on BM25 scores.
    """

    # 1. Split text into overlapping chunks
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap)]

    # 2. Tokenize the chunks
    tokenized_chunks = [re.findall(r"\w+", chunk) for chunk in chunks]

    # 3. Initialize BM25
    bm25 = BM25Okapi(tokenized_chunks)

    # 4. Query BM25 with the key question
    tokenized_question = re.findall(r"\w+", key_question)
    scores = bm25.get_scores(tokenized_question)

    # 5. Sort chunks by BM25 scores
    sorted_chunks = [chunk for _, chunk in sorted(zip(scores, chunks), key=lambda pair: pair[0], reverse=True)]

    # 6. Return top num_chunk_samples chunks
    return sorted_chunks


In [ ]:
# # DEBUG

# # Go through each full text rated highly to extract facts from
# full_text_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/sources/full_text'

# keywords_query = "ECR enzymes efficiency Kitsatospor setae bacteria" # Hardcoding for now, but from research_machine_v2

# with open(r'C:\Users\1kevi\Desktop\projects\Research\autoscious-carbon-capture\question_answering\autoscious_logs\How_efficiently_do_the_ECR_enzymes_work_in_Kitsatospor_setae_bacteria_\sources\full_text\Intersubunit_Coupling_Enables_Fast_CO2_Fixation_by_Reductive_Carboxylases.txt', 'r', encoding='utf-8') as file:
#     title = file.readline().strip()
#     url = file.readline().strip()
#     text = file.read()

# # chunks = get_most_relevant_chunks_specter(keywords_query, text, chunk_size)
# chunks = chunk_text(text, keywords_query, chunk_size, overlap)
# for i, chunk in enumerate(chunks):
#     print("i: ", i)
#     print(chunk)
#     print()

In [84]:
# This method can partially extract the answer, but not the exact table passage which is critical NOR does it prioritize the list of facts
# Go through each full text rated highly to extract facts from
full_text_folder_path = f'autoscious_logs/{sanitize_filename(search_query)}/sources/full_text'

keywords_query = "ECR enzymes efficiency Kitsatospor setae bacteria" # Hardcoding for now, but from research_machine_v2

# Loop through every file in the directory
for filename in os.listdir(full_text_folder_path):
    curr_tokens = 0
    if filename.endswith('.txt'):
        file_path = os.path.join(full_text_folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            title = file.readline().strip()
            url = file.readline().strip()
            text = file.read()
        
        # Break the text into chunk to extract information from
        chunks = chunk_text(text, keywords_query, chunk_size, overlap)
        for i, chunk in enumerate(chunks):
            print(f"Chunk: {i} / {len(chunks)}")
            extract_facts_from_website_text(search_query_file_safe, search_query, title, chunk, url)

            curr_tokens += len(chunk)
            if curr_tokens > MAX_TOKENS:
                print("Max tokens reached in chunks!")
                break

Chunk: 0 / 10
seed_initial_question_decomposition_prompt 
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. Format as a JSON.
```json
{
  "<insert best answer>": "<insert quote>"
}
```

Source text: ly behind. This is due to very diverse reasons including the overall very low growth rates of autotrophs and technical challenges such as difficulties supplying light or gaseous substrates in aqueous systems or the challenge of providing strictly oxygen-free cultivation conditions at a large scale for anaerobic microorganisms. The optimization of CO2-fixing organisms not only has to cope with the structural complexity of the multiprotein complexes responsible for photosynthesis and carbon capturing systems but also with 

In [ ]:
# OLD prompt: got partial answers, but never the direct answer
# Task: 
# Extract only direct quotes of facts or references from the source text that are 1) strong clues to answering the key question and 2) not less relevant than the existing list of best facts and references in answering the key question. Rank from most useful to least. Format as a JSON.
# ```json
# {{
#   "<insert unique quote index>": {{
#     "reasoning": "<one sentence explaining how this direct quote helps answer the key question>",
#     "quote": "<insert direct quote>",
#   }},
#   etc.
# }}
# ```

# Source text: {website_text}

# Respond only with the output, with no explanation or conversation.

In [78]:
# GPT template



prompt = f'''
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. Format as a JSON.
```json
{{
  "<insert best answer>": "<insert quote>"
}}
```

Source text: ing catalysis, how is catalysis synchronized across the enzyme complex (i.e., across dimers A/C and B/D)? One intriguing aspect of the ECR tetramer structures is that the catalytic domains share a common interface of 1636 Å2 between the pairs of dimers (Figure 5a,b), suggesting that they move together as rigid bodies.Figure 5Figure 5. Inter- and intradimer communications drive fast CO2 fixation by K. setae ECR. (a) Two distinct sets of communications: interdimer interactions between the catalytic domains from two dimers (purple arrows) and intradimer communication between the open and closed subunits within each dimer (brown arrows). (b) Interdimer catalytic domain interface and positions of selected amino acids that were mutated in this study to affect the interface between the two catalytic domains (open-form subunit in orange and closed form subunit in gray). The right panel shows the mutual H-bonding interaction between N218 and N157 from open- and closed-form subunits and H-bonding between E151 and the N atom from the protein backbone. (c) Alignment of ECR protein sequences from the primary (upper row) and the secondary (lower row) metabolisms represented as sequence logos. The numbering of residues, above the first row, is according to their position in the K. setae ECR. (d) Communication between the closed (green) and open (orange) subunits across the two dimers of the K. setae ECR. In the closed conformation the contacts between NADPH-H365-E165 and K332 of the adjacent open subunit allow for the correct intradimer communication. In the open conformation the communication network is compromised, as indicated by the increased distances between the amino acid side chains that cause the incorrect positioning of the nicotinamide ring of NADPH.High Resolution ImageDownload MS PowerPoint SlideWhat are the molecular determinants that synchronize catalysis across the pair of open/closed form dimers? The intercatalytic domain interface is predominantly hydrophobic with some electrostatic interactions (Figure 5b). Most notable are N218, which forms a hydrogen bond to N157 of the adjacent subunit of the other dimer, and E151 interacting with the backbone nitrogen atom of N133 (and/or A134) of the neighboring subunit of the other dimer (Figure 5b). A multiple sequence alignment using CLUSTAL W (20) showed that E151, N218, and N157 are highly conserved in ECRs from primary (i.e., central carbon) metabolism with faster CO2-fixation kinetics (average kcat 28 s–1), but not in ECRs from secondary metabolism with an average kcat of 1.2 s–1 (Figure 5c), which raised the question about their potential role in promoting catalysis in primary metabolic ECRs.Notably, mutation of these residues that are more than 20 Å away from the active site dramatically affected the kinetic parameters of K. setae ECR (Table 1). In the E151D variant, the kcat value decreased by a factor of 5, demonstrating that weakening the interaction between catalytic domains has profound effects on the enzyme’s catalytic rate. Mutations that targeted the asparagine interaction network also showed substantial effects on the catalytic rate and affected the KM value of substrate binding. Most notable were a N218E single variant and E151D/N157E/N218E triple variants that decreased the kcat values by more than 25- and 100-fold, respectively, highlighting that communication through the interface of the catalytic domains of the pair of dimers is an essential determinant of the catalytic rate in K. setae ECR.Table 1. Steady -State Analysis of K. setae ECR and Variants Targeting the Catalytic Domain Interface between the Pair of Dimersa crotonyl-CoANADPHCO2enzymeKM (μM)Ki (μM)kcat (s–1)KM (μM)kcat (s–1)KM (μM)kcat (s–1)wild type21 ± 23650 ± 810103 ± 337 ± 486 ± 290 ± 1078 ± 2E151D28 ± 21960 ± 25020 ± 172 ± 1117 ± 180 ± 1021 ± 1N157E515 ± 75 22 ± 1105 ± 2717.1 ± 0.440 ± 614 ± 0N218E272 ± 37 3.7 ± 0.266 ± 93.3 ± 0.0.2840 ± 1503.3 ± 0.2E151D/N157E/N218E245 ± 25 1.1 ± 0.026 ± 30.7 ± 0

Respond only with the output, with no explanation or conversation.
'''
res = chat_openai(prompt, model="gpt-3.5-turbo")[0]
print(res)

Prompt:  
Key question: 
How efficiently do the ECR enzymes work in Kitsatospor setae bacteria?

Context:
Ideally, I'm looking for a numerical answer.

Task:
What's the best answer based on the source text? Give me as specific and correct of an answer as possible. Then, quote the section of the source text that supports your answer. Format as a JSON.
```json
{
  "<insert best answer>": "<insert quote>"
}
```

Source text: ing catalysis, how is catalysis synchronized across the enzyme complex (i.e., across dimers A/C and B/D)? One intriguing aspect of the ECR tetramer structures is that the catalytic domains share a common interface of 1636 Å2 between the pairs of dimers (Figure 5a,b), suggesting that they move together as rigid bodies.Figure 5Figure 5. Inter- and intradimer communications drive fast CO2 fixation by K. setae ECR. (a) Two distinct sets of communications: interdimer interactions between the catalytic domains from two dimers (purple arrows) and intradimer communication bet